# Examine the Evalution Metrics

Examine the resulting model evaluation after the pipeline completes. Download the resulting evaluation.json file from S3 and print the report.

In [1]:
from botocore.exceptions import ClientError

import os
import sagemaker
import logging
import boto3
import sagemaker
import pandas as pd

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

In [2]:
%store -r pipeline_name

In [3]:
print(pipeline_name)

BERT-pipeline-1710454714


In [4]:
%store -r pipeline_experiment_name

In [5]:
print(pipeline_experiment_name)

BERT-pipeline-1710454714


In [6]:
%%time

import time
from pprint import pprint

executions_response = sm.list_pipeline_executions(PipelineName=pipeline_name)["PipelineExecutionSummaries"]
pipeline_execution_status = executions_response[0]["PipelineExecutionStatus"]
print(pipeline_execution_status)

while pipeline_execution_status == "Executing":
    try:
        executions_response = sm.list_pipeline_executions(PipelineName=pipeline_name)["PipelineExecutionSummaries"]
        pipeline_execution_status = executions_response[0]["PipelineExecutionStatus"]
    except Exception as e:
        print("Please wait...")
        time.sleep(30)

pprint(executions_response)

Succeeded
[{'PipelineExecutionArn': 'arn:aws:sagemaker:us-east-1:211125778552:pipeline/BERT-pipeline-1710454714/execution/s7paou48eed0',
  'PipelineExecutionDisplayName': 'execution-1710454725495',
  'PipelineExecutionStatus': 'Succeeded',
  'StartTime': datetime.datetime(2024, 3, 14, 22, 18, 45, 441000, tzinfo=tzlocal())}]
CPU times: user 16.4 ms, sys: 137 µs, total: 16.5 ms
Wall time: 241 ms


# List Pipeline Execution Steps

In [7]:
pipeline_execution_status = executions_response[0]["PipelineExecutionStatus"]
print(pipeline_execution_status)

Succeeded


In [8]:
pipeline_execution_arn = executions_response[0]["PipelineExecutionArn"]
print(pipeline_execution_arn)

arn:aws:sagemaker:us-east-1:211125778552:pipeline/BERT-pipeline-1710454714/execution/s7paou48eed0


In [9]:
from pprint import pprint

steps = sm.list_pipeline_execution_steps(PipelineExecutionArn=pipeline_execution_arn)

pprint(steps)

{'PipelineExecutionSteps': [{'AttemptCount': 1,
                             'EndTime': datetime.datetime(2024, 3, 14, 23, 0, 55, 235000, tzinfo=tzlocal()),
                             'Metadata': {'RegisterModel': {'Arn': 'arn:aws:sagemaker:us-east-1:211125778552:model-package/BERT-Reviews-1710454716/1'}},
                             'StartTime': datetime.datetime(2024, 3, 14, 23, 0, 54, 201000, tzinfo=tzlocal()),
                             'StepName': 'RegisterModel',
                             'StepStatus': 'Succeeded'},
                            {'AttemptCount': 1,
                             'EndTime': datetime.datetime(2024, 3, 14, 23, 0, 55, 344000, tzinfo=tzlocal()),
                             'Metadata': {'Model': {'Arn': 'arn:aws:sagemaker:us-east-1:211125778552:model/pipelines-s7paou48eed0-createmodel-dtxkpwxj4c'}},
                             'StartTime': datetime.datetime(2024, 3, 14, 23, 0, 54, 201000, tzinfo=tzlocal()),
                             'StepName'

# Retrieve Evaluation Metrics

In [10]:
# for execution_step in reversed(execution.list_steps()):
for execution_step in reversed(steps["PipelineExecutionSteps"]):
    if execution_step["StepName"] == "EvaluateModel":
        processing_job_name = execution_step["Metadata"]["ProcessingJob"]["Arn"].split("/")[-1]

describe_evaluation_processing_job_response = sm.describe_processing_job(ProcessingJobName=processing_job_name)

evaluation_metrics_s3_uri = describe_evaluation_processing_job_response["ProcessingOutputConfig"]["Outputs"][0][
    "S3Output"
]["S3Uri"]
evaluation_metrics_s3_uri

's3://sagemaker-us-east-1-211125778552/sagemaker-scikit-learn-2024-03-14-22-18-40-486/output/metrics'

In [11]:
import json
from pprint import pprint

evaluation_json = sagemaker.s3.S3Downloader.read_file("{}/evaluation.json".format(evaluation_metrics_s3_uri))

pprint(json.loads(evaluation_json))

{'metrics': {'accuracy': {'value': 0.56}}}


# Download and Analyze the Trained Model from S3

In [12]:
training_job_arn = None

for execution_step in steps["PipelineExecutionSteps"]:
    if execution_step["StepName"] == "Train":
        training_job_arn = execution_step["Metadata"]["TrainingJob"]["Arn"]

        break

training_job_name = training_job_arn.split("/")[-1]
print(training_job_name)

pipelines-s7paou48eed0-Train-jZKKoTNirt


In [13]:
model_tar_s3_uri = sm.describe_training_job(TrainingJobName=training_job_name)["ModelArtifacts"]["S3ModelArtifacts"]

In [14]:
!aws s3 cp $model_tar_s3_uri ./

download: s3://sagemaker-us-east-1-211125778552/pipelines-s7paou48eed0-Train-jZKKoTNirt/output/model.tar.gz to ./model.tar.gz


In [15]:
!mkdir -p ./model
!tar -zxvf model.tar.gz -C ./model

tar: Ignoring unknown extended header keyword 'LIBARCHIVE.creationtime'
transformers/
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.creationtime'
transformers/fine-tuned/
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.creationtime'
transformers/fine-tuned/tf_model.h5
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.creationtime'
transformers/fine-tuned/config.json
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.creationtime'
code/
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.creationtime'
code/inference.py
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.creationtime'
tensorboard/
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.creationtime'
test_data/
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.creationtime'
test_data/amazon_reviews_us_Digital_Software_v1_00.tsv.gz
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.creationtime'
tensorflow/
tar: Ignoring unknown extended header keyword 'LIBA

In [16]:
!saved_model_cli show --all --dir ./model/tensorflow/saved_model/0/

2024-03-14 23:21:06.596090: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2024-03-14 23:21:06.596544: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.

MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['input_ids'] tensor_info:
        dtype: DT_INT32
        shape: (-1, 64)
        name: serving_defau

In [17]:
!saved_model_cli run --dir ./model/tensorflow/saved_model/0/ --tag_set serve --signature_def serving_default \
    --input_exprs 'input_ids=np.zeros((1,64));input_mask=np.zeros((1,64))'

2024-03-14 23:21:22.735976: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2024-03-14 23:21:22.736009: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-03-14 23:21:24.579890: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-03-14 23:21:24.579933: W tensorflow/stream_executor/cuda/cuda_driver.cc:312] failed call to cuInit: UNKNOWN ERROR (303)
2024-03-14 23:21:24.579956: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (default): /proc/driver/nvidia/version does not exist
2024-03-14 23:21:24.580468: I tensorflow/core/platform/cpu_feature_gu

# List All Artifacts Generated By The Pipeline

In [18]:
processing_job_name = None
training_job_name = None

In [19]:
import time
from sagemaker.lineage.visualizer import LineageTableVisualizer

viz = LineageTableVisualizer(sagemaker.session.Session())

for execution_step in reversed(steps["PipelineExecutionSteps"]):
    print(execution_step)
    # We are doing this because there appears to be a bug of this LineageTableVisualizer handling the Processing Step
    if execution_step["StepName"] == "Processing":
        processing_job_name = execution_step["Metadata"]["ProcessingJob"]["Arn"].split("/")[-1]
        print(processing_job_name)
        display(viz.show(processing_job_name=processing_job_name))
    elif execution_step["StepName"] == "Train":
        training_job_name = execution_step["Metadata"]["TrainingJob"]["Arn"].split("/")[-1]
        print(training_job_name)
        display(viz.show(training_job_name=training_job_name))
    else:
        display(viz.show(pipeline_execution_step=execution_step))
        time.sleep(5)

{'StepName': 'Processing', 'StartTime': datetime.datetime(2024, 3, 14, 22, 18, 46, 76000, tzinfo=tzlocal()), 'EndTime': datetime.datetime(2024, 3, 14, 22, 32, 31, 313000, tzinfo=tzlocal()), 'StepStatus': 'Succeeded', 'AttemptCount': 1, 'Metadata': {'ProcessingJob': {'Arn': 'arn:aws:sagemaker:us-east-1:211125778552:processing-job/pipelines-s7paou48eed0-Processing-yEvdntPfGq'}}}
pipelines-s7paou48eed0-Processing-yEvdntPfGq


,Name/Source,Direction,Type,Association Type,Lineage Type
0,s3://...ess-scikit-text-to-bert-feature-store.py,Input,DataSet,ContributedTo,artifact
1,s3://...t-1-211125778552/amazon-reviews-pds/tsv/,Input,DataSet,ContributedTo,artifact
2,68331...om/sagemaker-scikit-learn:0.23-1-cpu-py3,Input,Image,ContributedTo,artifact
3,s3://...2024-03-14-22-18-41-139/output/bert-test,Output,DataSet,Produced,artifact
4,s3://...3-14-22-18-41-139/output/bert-validation,Output,DataSet,Produced,artifact
5,s3://...024-03-14-22-18-41-139/output/bert-train,Output,DataSet,Produced,artifact


{'StepName': 'Train', 'StartTime': datetime.datetime(2024, 3, 14, 22, 32, 32, 350000, tzinfo=tzlocal()), 'EndTime': datetime.datetime(2024, 3, 14, 22, 48, 6, 591000, tzinfo=tzlocal()), 'StepStatus': 'Succeeded', 'AttemptCount': 1, 'Metadata': {'TrainingJob': {'Arn': 'arn:aws:sagemaker:us-east-1:211125778552:training-job/pipelines-s7paou48eed0-Train-jZKKoTNirt'}}}
pipelines-s7paou48eed0-Train-jZKKoTNirt


,Name/Source,Direction,Type,Association Type,Lineage Type
0,s3://...2024-03-14-22-18-41-139/output/bert-test,Input,DataSet,ContributedTo,artifact
1,s3://...3-14-22-18-41-139/output/bert-validation,Input,DataSet,ContributedTo,artifact
2,s3://...024-03-14-22-18-41-139/output/bert-train,Input,DataSet,ContributedTo,artifact
3,76310...s.com/tensorflow-training:2.3.1-cpu-py37,Input,Image,ContributedTo,artifact
4,s3://...ed0-Train-jZKKoTNirt/output/model.tar.gz,Output,Model,Produced,artifact


{'StepName': 'EvaluateModel', 'StartTime': datetime.datetime(2024, 3, 14, 22, 48, 7, 713000, tzinfo=tzlocal()), 'EndTime': datetime.datetime(2024, 3, 14, 23, 0, 52, 365000, tzinfo=tzlocal()), 'StepStatus': 'Succeeded', 'AttemptCount': 1, 'Metadata': {'ProcessingJob': {'Arn': 'arn:aws:sagemaker:us-east-1:211125778552:processing-job/pipelines-s7paou48eed0-EvaluateModel-U8RiAVLfZd'}}}


...

None

{'StepName': 'CreateModel', 'StartTime': datetime.datetime(2024, 3, 14, 23, 0, 54, 201000, tzinfo=tzlocal()), 'EndTime': datetime.datetime(2024, 3, 14, 23, 0, 55, 344000, tzinfo=tzlocal()), 'StepStatus': 'Succeeded', 'AttemptCount': 1, 'Metadata': {'Model': {'Arn': 'arn:aws:sagemaker:us-east-1:211125778552:model/pipelines-s7paou48eed0-createmodel-dtxkpwxj4c'}}}


None

{'StepName': 'RegisterModel', 'StartTime': datetime.datetime(2024, 3, 14, 23, 0, 54, 201000, tzinfo=tzlocal()), 'EndTime': datetime.datetime(2024, 3, 14, 23, 0, 55, 235000, tzinfo=tzlocal()), 'StepStatus': 'Succeeded', 'AttemptCount': 1, 'Metadata': {'RegisterModel': {'Arn': 'arn:aws:sagemaker:us-east-1:211125778552:model-package/BERT-Reviews-1710454716/1'}}}


,Name/Source,Direction,Type,Association Type,Lineage Type
0,s3://...ed0-Train-jZKKoTNirt/output/model.tar.gz,Input,Model,ContributedTo,artifact
1,76310...onaws.com/tensorflow-inference:2.3.1-cpu,Input,Image,ContributedTo,artifact
2,BERT-Reviews-1710454716-1-PendingManualApprova...,Input,Approval,ContributedTo,action
3,BERT-Reviews-1710454716-1710457254-aws-model-p...,Output,ModelGroup,AssociatedWith,context


# Analyze Experiment

In [20]:
from sagemaker.analytics import ExperimentAnalytics

time.sleep(30)  # avoid throttling exception

import pandas as pd

pd.set_option("max_colwidth", 500)

experiment_analytics = ExperimentAnalytics(
    experiment_name=pipeline_experiment_name,
)

experiment_analytics.dataframe()

,TrialComponentName,DisplayName,SourceArn,AWS_DEFAULT_REGION,SageMaker.InstanceCount,SageMaker.InstanceType,SageMaker.VolumeSizeInGB,balance_dataset,feature_group_name,feature_store_offline_prefix,...,test - MediaType,test - Value,train - MediaType,train - Value,validation - MediaType,validation - Value,SageMaker.DebugHookOutput - MediaType,SageMaker.DebugHookOutput - Value,SageMaker.ModelArtifact - MediaType,SageMaker.ModelArtifact - Value
0,pipelines-s7paou48eed0-Processing-yEvdntPfGq-aws-processing-job,pipelines-s7paou48eed0-Processing-yEvdntPfGq-aws-processing-job,arn:aws:sagemaker:us-east-1:211125778552:processing-job/pipelines-s7paou48eed0-Processing-yEvdntPfGq,us-east-1,1.0,ml.c5.2xlarge,30.0,True,reviews-feature-group-1710454716,reviews-feature-store-1710454716,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,pipelines-s7paou48eed0-Train-jZKKoTNirt-aws-training-job,pipelines-s7paou48eed0-Train-jZKKoTNirt-aws-training-job,arn:aws:sagemaker:us-east-1:211125778552:training-job/pipelines-s7paou48eed0-Train-jZKKoTNirt,NaN,1.0,ml.c5.9xlarge,1024.0,NaN,NaN,NaN,...,text/csv,s3://sagemaker-us-east-1-211125778552/sagemaker-scikit-learn-2024-03-14-22-18-41-139/output/bert-test,text/csv,s3://sagemaker-us-east-1-211125778552/sagemaker-scikit-learn-2024-03-14-22-18-41-139/output/bert-train,text/csv,s3://sagemaker-us-east-1-211125778552/sagemaker-scikit-learn-2024-03-14-22-18-41-139/output/bert-validation,NaN,s3://sagemaker-us-east-1-211125778552,NaN,s3://sagemaker-us-east-1-211125778552/pipelines-s7paou48eed0-Train-jZKKoTNirt/output/model.tar.gz


# Release Resources

In [21]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>

<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [22]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}

<IPython.core.display.Javascript object>